In [5]:
import pandas as pd
import numpy as np
import jieba
import logging
import fasttext

In [7]:
train = pd.read_csv('train.tsv', sep='\t', encoding='gb18030')
test = pd.read_csv('test.tsv', sep='\t', encoding='gb18030')

x = train['ITEM_NAME']
y = train['TYPE']

train.head(10)

,ITEM_NAME,TYPE
0,腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费,本地生活--游戏充值--QQ充值
1,腾讯q币充值30元qq币30qb30q币自动充值,本地生活--游戏充值--QQ充值
2,腾讯QQ红钻包月卡 QQ红钻一个月QQ红钻1个月,本地生活--游戏充值--QQ充值
3,腾讯QQ蓝钻贵族31个月直充,本地生活--游戏充值--QQ充值
4,腾讯QQ币148元148QQ币148个直充148Q币148个Q币148个QQB★自动充值,本地生活--游戏充值--QQ充值
5,可升级2016MAC 微软原装正版office 365个人版续费 开发票请拍实物寄送 1PC...,本地生活--游戏充值--QQ充值
6,腾讯QQ币419元419QQ币自动充值419qb/419个QQB★自动充值,本地生活--游戏充值--QQ充值
7,腾讯QQ黑钻贵族九个月 地下城勇士黑钻9个月 包月卡可查时间可续费★自动充值,本地生活--游戏充值--QQ充值
8,腾讯QQ币457元 457QB 457QQ币457个 457Q币 457个Q币 自动充值,本地生活--游戏充值--QQ充值
9,QQ音乐绿钻豪华版四个月 豪华绿钻绿钻豪华版4个月 自动充值,本地生活--游戏充值--QQ充值


In [8]:
# 数据打乱，切分训练集和测试集
def train_test_split(x, y, seed = 10):
    np.random.seed(seed)
    shuffle_indices = np.random.permutation(np.arange(len(x)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    x_train = x_shuffled[:int(len(x)*0.95)]
    y_train = y_shuffled[:int(len(x)*0.95)]

    x_test = x_shuffled[int(len(x)*0.95):]
    y_test = y_shuffled[int(len(x)*0.95):]
    
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    return x_train, y_train, x_test, y_test

# 写标准fasttext标准文件
def data_write(filename, x, y):
    with open(filename, 'w') as f:
        for i, doc in enumerate(x):
            doc = jieba.cut(doc.strip(), cut_all=False)
            doc = ' '.join(list(doc))
            predix = '__label__'
            label = predix + str(y[i])
            line = doc + '\t' + label + '\n'
            f.write(line)
            

In [9]:
x = np.array(train['ITEM_NAME'])
y = np.array(train['TYPE'])
x_train, y_train, x_test, y_test = train_test_split(x, y)
data_write('train_all.txt', x_train, y_train)
data_write('test_all.txt', x_test, y_test)

Building prefix dict from the default dictionary ...
2019-03-14 10:59:16,142 : DEBUG : Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
2019-03-14 10:59:16,152 : DEBUG : Loading model from cache /tmp/jieba.cache
Loading model cost 1.330 seconds.
2019-03-14 10:59:17,479 : DEBUG : Loading model cost 1.330 seconds.
Prefix dict has been built succesfully.
2019-03-14 10:59:17,481 : DEBUG : Prefix dict has been built succesfully.


In [11]:
classifier = fasttext.supervised('train_all.txt', 'train_all.model', label_prefix='__label__')
# classifier = fasttext.load_model('train_all.model.bin', label_prefix='__label__')
result = classifier.test('test_all.txt')
precision = result.precision
recall = result.recall
length = len(x)
print('样本数量：%d; 精确率：%f; 回报率：%f' % (length, precision, recall))

样本数量：499447; 精确率：0.819966; 回报率：0.819966


In [13]:
pre_labels = []
for line in test['ITEM_NAME'][:10]:
    line = jieba.cut(line.strip(), cut_all=False)
    line = [str(' '.join(list(line)))]
    label = classifier.predict(line)[0][0]
    pre_labels.append(label)
    print(line, label)

['腾讯 QQ 蓝 钻 贵族 34 个 月 直充'] 服饰鞋帽--女鞋--单鞋
['腾讯 QQ 币 24 元 24QQ 币 24 个 直充 24Q 币 24 个 Q 币 24 个 QB / 24QQB ★ 自动 充值'] 本地生活--游戏充值--游戏点卡
['腾讯 QQ 币 101 元 101QQ 币 101 个 直充 101Q 币 101 个 Q 币 101 个 QB ★ 自动 充值'] 本地生活--游戏充值--游戏点卡
['腾讯 QQ 币 402 元   402QB   402QQ 币 402 个   402Q 币   402 个 Q 币   自动 充值'] 本地生活--游戏充值--游戏点卡
['腾讯 QQ 币 380 元   380QB   380QQ 币 380 个   380Q 币   380 个 Q 币   自动 充值'] 本地生活--游戏充值--游戏点卡
['腾讯 QQ 币 424 元   424QB   424QQ 币 424 个   424Q 币   424 个 Q 币   自动 充值'] 本地生活--游戏充值--游戏点卡
['腾讯 QQ 币 479 元   479QB   479QQ 币 479 个   479Q 币   479 个 Q 币   自动 充值'] 本地生活--游戏充值--游戏点卡
['腾讯 QQ 会员 1 个 月 QQ 会员 一个月 包月卡 自动 充值 可 查 时间 可 续费'] 本地生活--游戏充值--游戏点卡
['腾讯 QQ 蓝 钻 包月卡   QQ 蓝 钻 一个月 QQ 蓝 钻 1 个 月   1 个 月 QQ 蓝 钻   可查 可续   自动 充值'] 服饰鞋帽--女鞋--单鞋
['腾讯 QQ 币 35 元 35QQ 币 35 个 直充 35Q 币 35 个 Q 币 35 个 QB / 35QQB ★ 自动 充值'] 服饰鞋帽--女鞋--单鞋
